In [1]:
# Parameters
end_period = "2015-06-30"
smoothing = True


## HLW_FC

In [2]:
from bok_da.ts.nir import HLW_FC

In [3]:
# !pip install openpyxl
import pandas as pd
import numpy as np
import datetime

In [4]:
KORData = pd.read_excel("data/nir_data_all.xlsx", sheet_name="raw_data")
KORData['creditngdp'] = np.log(KORData['credit']/KORData['ngdp'])*100
KORData['creditratio'] = KORData['creditngdp'][4:] - KORData['creditngdp'].shift(4)
KORData

,Date,rgdp,inf1,inf2,infe,interest,ngdp,credit,csi,ci,creditngdp,creditratio
0,1960-01-01,7530.1,NaN,NaN,NaN,NaN,56.64,NaN,0.0,0,NaN,NaN
1,1960-04-01,7878.4,NaN,NaN,NaN,NaN,61.53,NaN,0.0,0,NaN,NaN
2,1960-07-01,7788.8,NaN,NaN,NaN,NaN,65.44,NaN,0.0,0,NaN,NaN
3,1960-10-01,7834.3,NaN,NaN,NaN,NaN,67.13,NaN,0.0,0,NaN,NaN
4,1961-01-01,8081.8,NaN,NaN,NaN,NaN,72.93,NaN,0.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
255,2023-10-01,565597.6,111.30,109.47,3.33,3.576,613826.20,1885451.2,0.0,0,112.221061,-4.543617
256,2024-01-01,572951.0,111.90,110.26,3.07,3.506,632142.20,1882385.4,0.0,0,109.118071,-5.487701
257,2024-04-01,571646.7,112.39,110.84,3.10,3.532,638253.40,1895802.0,0.0,0,108.866186,-5.140436
258,2024-07-01,572216.0,112.76,111.28,2.87,3.526,633042.30,1913814.3,0.0,0,110.631630,-2.986281


In [5]:
end_date = datetime.datetime.strptime('2024-07-01', '%Y-%m-%d')
KORData_before = KORData[KORData['Date'] <= end_date]
data = KORData_before.set_index("Date")
data_list = ['rgdp', 'inf1', 'interest', 'infe', 'creditratio']
data = data[data_list]
data['inf1'] = data['inf1'].pct_change(4) * 100

if smoothing:
    data['inf1'] = data['inf1'].rolling(4).mean() # 4분기 이동평균
    
data['rgdp'] = np.log(data['rgdp']) * 400
data['interest'] = data['interest'] - data['inf1']
data = data.drop('infe', axis=1)
data = data.dropna()
#data = data.loc['1999-01-01':]
KORmacro = data['2000-01-01':].copy()
#KORmacro = data.copy()
def shift_quarter_end(date):
    if date.month == 1:
        return date.replace(month=3, day=31)
    elif date.month == 4:
        return date.replace(month=6, day=30)
    elif date.month == 7:
        return date.replace(month=9, day=30)
    else:
        return date.replace(month=12, day=31)

KORmacro.index = KORmacro.index.map(shift_quarter_end)
KORmacro

,rgdp,inf1,interest,creditratio
Date,,,,
2003-12-31,5042.156092,3.063351,0.706649,-5.628428
2004-03-31,5047.808057,2.946573,0.823427,-6.439517
2004-06-30,5051.129892,2.831088,0.928912,-6.507453
2004-09-30,5052.151916,2.837248,0.792752,-5.050434
2004-12-31,5055.417125,2.924617,0.455383,-1.876856
...,...,...,...,...
2023-09-30,5296.439326,4.367675,-0.819675,-3.340593
2023-12-31,5298.255260,4.007923,-0.431923,-4.543617
2024-03-31,5303.422191,3.466374,0.039626,-5.487701


In [6]:
mcmc = {
    'sig2_g': 0,
    'sig2_yt': 0,
    'sig2_yc': 0,
    'sig2_z': 0,
    'sig2_p': 0,
    'slope_yc': 0,
    'slope_z': 0,
    'slope_p': 0,
    'sig2_f': 0,
    'slope_f': 0
}

prior = {
    'yc_mean': np.array([0.7, 0.2, -0.3]),
    'yc_variance': np.diag(np.array([0.1, 0.1, 0.03]) ** 2),
    'sig2_yc_shape': 5,
    'sig2_yc_scale': 4 * 2**2,
    'sig2_g_shape': 5,
    'sig2_g_scale': 4 * 0.1**2,
    'sig2_yt_shape': 5,
    'sig2_yt_scale': 4 * 2**2,
    'sig2_z_shape': 5,
    'sig2_z_scale': 4 * 0.5**2,
    'p_mean': np.array([0.7, 0.1]),
    'p_variance': np.diag(np.array([0.1, 0.1]) ** 2),
    'sig2_p_shape': 5,
    'sig2_p_scale': 4 * 0.1**2,
    'f_mean': np.array([0, 0.7, 0.2, -0.3]),
    'f_variance': np.diag(np.array([0.1, 0.5, 0.5, 0.05]) ** 2),
    'sig2_f_shape': 5,
    'sig2_f_scale': 4 * 0.1**2
}

model = HLW_FC(n_draws=5000, burnin=1000, MCMCAlgorithm=mcmc, PriorPara=prior, ZSpec='rw')

> Initializing Model...
  - Total MCMC iterations: 5000
  - Burn-in period: 1000
  - Latent variables percentiles: [5, 95]
  - Parameters percentiles: [0.05, 0.95]
  - Interest rate z-component specification: rw


In [7]:
model.fit(KORmacro.loc[:end_period], dates=KORmacro.loc[:end_period].index)

> Starting model fitting...
  - Data shape: (47, 4)
  - Dates provided with length: 47


Sampling:   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 0/5000 [00:00<?, ?it/s]

Step 1/3 :   0%|          | 0/5000 [00:00<?, ?it/s]

...

Step 3/3 :  46%|████▌     | 2282/5000 [00:29<00:25, 107.16it/s]

Step 3/3 :  46%|████▌     | 2282/5000 [00:29<00:25, 107.16it/s]

Step 3/3 :  46%|████▌     | 2282/5000 [00:29<00:25, 107.16it/s]

Step 3/3 :  46%|████▌     | 2282/5000 [00:29<00:25, 107.16it/s]

Step 3/3 :  46%|████▌     | 2282/5000 [00:29<00:25, 107.16it/s]

In [ ]:
rstar_HLW_FC = pd.DataFrame(model.SummaryLV['Avg']['rNatural'])
rstar_HLW_FC.index = KORmacro.loc[KORmacro.index[4]:end_period].index
if smoothing:
    rstar_HLW_FC.columns = ['rstar_HLW_FC_smoothed']
else:
    rstar_HLW_FC.columns = ['rstar_HLW_FC']
rstar = rstar_HLW_FC.reset_index().rename(columns={'index': 'Date'}).to_json(orient='records', date_format='iso')
rstar_HLW_FC.reset_index().rename(columns={'index': 'Date'}) \
    .assign(Date=lambda df: df['Date'].dt.strftime('%Y-%m-%d')) \
    .to_json('rstar_HLW_FC.json', orient='records')